In [2]:
library(tidyverse)
library(tidymodels)
library(readxl)
library(cowplot)
options(repr.matrix.rows=6)


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [3]:
training_data <- read_csv("Training_KAHRAMAN.csv")
testing_data <- read_csv("Testing_KAHRAMAN.csv")

Warning message:
“Missing column names filled in: 'X7' [7], 'X8' [8]”
Parsed with column specification:
cols(
  STG = col_double(),
  SCG = col_double(),
  STR = col_double(),
  LPR = col_double(),
  PEG = col_double(),
  UNS = col_character(),
  X7 = col_logical(),
  X8 = col_logical(),
  `Attribute Information:` = col_character()
)

Warning message:
“Missing column names filled in: 'X7' [7], 'X8' [8]”
Parsed with column specification:
cols(
  STG = col_double(),
  SCG = col_double(),
  STR = col_double(),
  LPR = col_double(),
  PEG = col_double(),
  UNS = col_character(),
  X7 = col_logical(),
  X8 = col_logical(),
  `Attribute Information:` = col_character()
)



In [4]:
training_tidy <- training_data %>%
                    select(STG, SCG, STR, LPR, PEG, UNS) %>%
                    mutate(UNS = as_factor(UNS))
testing_tidy <- testing_data %>%
                    select(STG, SCG, STR, LPR, PEG, UNS)%>%
                    mutate(UNS = as_factor(UNS))
training_summary <- training_tidy %>%
                        summarise(avg_STG = mean(STG),
                                  avg_SCG = mean(SCG), 
                                  avg_STR = mean(STR), 
                                  avg_LPR = mean(LPR), 
                                  avg_PEG = mean(PEG))
head(training_tidy)
head(testing_tidy)
training_summary

STG,SCG,STR,LPR,PEG,UNS
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
0.00,0.00,0.00,0.00,0.00,very_low
0.08,0.08,0.10,0.24,0.90,High
0.06,0.06,0.05,0.25,0.33,Low
0.10,0.10,0.15,0.65,0.30,Middle
0.08,0.08,0.08,0.98,0.24,Low
0.09,0.15,0.40,0.10,0.66,Middle


STG,SCG,STR,LPR,PEG,UNS
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
0.00,0.10,0.50,0.26,0.05,Very Low
0.05,0.05,0.55,0.60,0.14,Low
0.08,0.18,0.63,0.60,0.85,High
0.20,0.20,0.68,0.67,0.85,High
0.22,0.22,0.90,0.30,0.90,High
0.14,0.14,0.70,0.50,0.30,Low


avg_STG,avg_SCG,avg_STR,avg_LPR,avg_PEG
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.3711473,0.3556744,0.4680039,0.4327132,0.4585388


In [24]:
#Evaluate classification accuracy using appropriate metrics
predictions_met <-class_prediction %>%
  metrics(truth =UNS, estimate = .pred_class)

predictions_conf <-class_prediction %>%
  conf_mat(truth =UNS, estimate = .pred_class)

ERROR: Error: In metric: `accuracy`
Problem with `summarise()` input `.estimate`.
[31m✖[39m `truth` and `estimate` levels must be equivalent.
`truth`: Very Low, Low, High, Middle
`estimate`: very_low, High, Low, Middle

[34mℹ[39m Input `.estimate` is `metric_fn(truth = UNS, estimate = .pred_class, na_rm = na_rm)`.


In [25]:
#corss-validation accuracy

uns_vfold <- vfold_cv(training_tidy, v = 10, strata = UNS)
vfold_metrics <- workflow() %>%
                  add_recipe(uns_recipe) %>%
                  add_model(knn_spec) %>%
                  fit_resamples(resamples = uns_vfold) %>%
                  collect_metrics()
vfold_metrics

x Fold01: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold02: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold03: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold04: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold05: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold06: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold07: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold08: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold09: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold10: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

Warning message:
“All models failed in [fit_resamples()]. See the `.notes` column.”


ERROR: Error: All of the models failed. See the .notes column.


In [22]:
#corss-validation accuracy
uns_vfold <- vfold_cv(training_tidy, v = 10, strata = UNS)
vfold_metrics <- workflow() %>%
                  add_recipe(uns_recipe) %>%
                  add_model(knn_spec) %>%
                  fit_resamples(resamples = uns_vfold) %>%
                  collect_metrics()
vfold_metrics

#cross-validation to choose K classification
set.seed(2020)
uns_recipe <- recipe(UNS ~LPR+PEG, data = training_tidy) %>%
  step_scale(all_predictors()) %>%
  step_center(all_predictors())

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
  set_engine("kknn") %>%
  set_mode("classification")


knn_results <- workflow() %>%
  add_recipe(uns_recipe) %>%
  add_model(knn_spec) %>%
  tune_grid(resamples = uns_vfold, grid = 10) %>%
  collect_metrics()
knn_results

accuracies <- knn_results %>%
  filter(.metric == "accuracy")
accuracy_vs_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", y = "Accuracy Estimate")
accuracy_vs_k

x Fold01: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold02: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold03: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold04: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold05: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold06: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold07: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold08: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold09: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

x Fold10: model: Error in sort.int(x, na.last = na.last, decreasing = decreasing, ...

Warning message:
“All models failed in [fit_resamples()]. See the `.notes` column.”


ERROR: Error: All of the models failed. See the .notes column.
